# 薪资数据分析

本Notebook用于分析薪资数据，包含数据加载、处理和可视化的步骤。

## 读取数据

导入pandas数据库，并通过read_csv函数，将原始数据文件"salary_data"里的数据内容，解析为DataFrame，并赋值给变量original_data。

In [37]:
import pandas as pd

In [38]:
original_data = pd.read_csv('../data/salary_data.csv')
original_data.tail()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
6699,49.0,Female,PhD,Director of Marketing,20.0,200000.0
6700,32.0,Male,High School,Sales Associate,3.0,50000.0
6701,30.0,Female,Bachelor's Degree,Financial Manager,4.0,55000.0
6702,46.0,Male,Master's Degree,Marketing Manager,14.0,140000.0
6703,26.0,Female,High School,Sales Executive,1.0,35000.0


## 评估数据

主要对上一部分建立的original所包含的数据进行评估。

评估主要从整齐度和干净度，即结构和内容两个方面进行。结构性：不符合"每列是一个变量，每行是一个观察值，每个单元格是一个值"这三个标准，内容行：丢失数据、重复数据、无效数据等。

### 评估数据整齐度

In [39]:
original_data.sample(10)

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
3766,44.0,Male,PhD,Data Scientist,15.0,150000.0
6121,29.0,Male,Bachelor's Degree,Marketing Analyst,4.0,75000.0
3175,34.0,Male,Master's Degree,Data Scientist,9.0,140000.0
2475,30.0,Male,High School,Senior Project Engineer,11.0,103947.0
5736,27.0,Male,Bachelor's Degree,Software Engineer,3.0,80000.0
1019,25.0,Female,Bachelor's,Data Analyst,2.0,100000.0
6511,35.0,Female,PhD,Senior Product Marketing Manager,9.0,95000.0
4738,38.0,Female,PhD,Senior Research Scientist,12.0,140000.0
2479,35.0,Female,Master's Degree,Full Stack Engineer,10.0,152039.0
1845,46.0,Male,PhD,Senior Project Engineer,15.0,180000.0


从抽样的10行数据来看，数据符合结构性要求，每行是某人的具体信息，每列是信息的各个变量，不存在结构性问题

### 评估数据干净度

In [40]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6702 non-null   float64
 1   Gender               6702 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6702 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6699 non-null   float64
dtypes: float64(3), object(3)
memory usage: 314.4+ KB


从结果来看，每一列都存在缺失值，年龄、工作经验和薪水都是浮点数，性别，教育水平和职位都是object，类型没有出错。因此下一步分析具体是那些行数据缺失，看是否可以根据其他相关数据进行补充。

### 评估缺失数据

In [41]:
original_data[(original_data['Salary'].isnull()) | (original_data['Education Level'].isnull())]

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
172,NaN,NaN,NaN,NaN,NaN,NaN
260,NaN,NaN,NaN,NaN,NaN,NaN
2011,27.0,Male,NaN,Developer,7.0,100000.0
3136,31.0,Male,Master's Degree,Full Stack Engineer,8.0,NaN
5247,26.0,Female,Bachelor's Degree,Social M,NaN,NaN
6455,36.0,Male,Bachelor's Degree,Sales Director,6.0,NaN


根据观察，可以看出，以上的6行数据缺失即为缺失的全部数据，其中172、260及5247行可删除，3136及6455可根据同样的职位、教育水平、级别和工作年限取平均值

2011行的Education Level可以根据其他同样的Gender,Job Title及Years of Experience的人来进行评估

In [42]:
original_data[(original_data['Years of Experience'] >= 7) & 
              (original_data['Salary'] >= 90000) & 
              (original_data['Salary'] <= 110000) & 
              (original_data['Gender'] == 'Male')]['Education Level'].mode()[0]

"Master's Degree"

可以把2011行的Education Level修改为"Master's Degree"

### 评估重复数据

根据观察，所有数据都可以重复，无需评估重复数据。

### 评估不一致数据

不一致数据可能存在Education Level中，需要查看是否有多个不同值代指同一个教育等级的情况。

In [43]:
original_data['Education Level'].value_counts()

Education Level
Bachelor's Degree    2267
Master's Degree      1573
PhD                  1368
Bachelor's            756
High School           448
Master's              288
phD                     1
Name: count, dtype: int64

从Education Level列中可以看到，"Bachelor's Degree"和"Bachelor's"表示同一学历等级，"PhD"和"phD"表示同一学历等级，"Master's Degree"和"Master's"也表示同一学历等级，因此需要将它们进行统一。

### 评估无效或错误数据

In [44]:
original_data.describe()

,Age,Years of Experience,Salary
count,6702.000000,6701.000000,6699.000000
mean,33.620859,8.094687,115326.964771
std,7.614633,6.059003,52786.183911
min,21.000000,0.000000,350.000000
25%,28.000000,3.000000,70000.000000
50%,32.000000,7.000000,115000.000000
75%,38.000000,12.000000,160000.000000
max,62.000000,34.000000,250000.000000


## 清理数据

根据前面评估部分，我们需要的进行的数据清理包括：
* 删除172、260及5247行，3136及6455可根据同样的职位、教育水平、级别和工作年限取平均值
* 可以把2011行的Education Level修改为"Master's Degree"
* 把Education Level变量值"Bachelor's"替换成"Bachelor's Degree"
* 把Education Level变量值"phD"替换成"PhD"
* 把Education Level变量值"Master's"替换成"Master's Degree"

In [45]:
original_data.drop(index=[172, 260, 5247], inplace=True)
original_data.loc[3136, 'Salary'] = original_data[(original_data['Education Level'] == "Master's Degree") & (original_data['Job Title'] == 'Full Stack Engineer') & (original_data['Years of Experience'] == 8.0)]['Salary'].mean()
original_data.loc[6455, 'Salary'] = original_data[(original_data['Education Level'] == "Bachelor's Degree") & (original_data['Job Title'] == 'Sales Director') & (original_data['Years of Experience'] == 6.0)]['Salary'].mean()
original_data.loc[2011, 'Education Level'] = "Master's Degree"
original_data['Education Level'].replace({'Bachelor\'s':"Bachelor's Degree", 'phD':'PhD', "Master's":"Master's Degree"}, inplace=True)
original_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6701 entries, 0 to 6703
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6701 non-null   float64
 1   Gender               6701 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6701 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6701 non-null   float64
dtypes: float64(3), object(3)
memory usage: 624.5+ KB


C:\Users\Ammer\AppData\Local\Temp\ipykernel_23988\3896847141.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  original_data['Education Level'].replace({'Bachelor\'s':"Bachelor's Degree", 'phD':'PhD', "Master's":"Master's Degree"}, inplace=True)


In [47]:
original_data['Education Level'].value_counts()

Education Level
Bachelor's Degree    3022
Master's Degree      1862
PhD                  1369
High School           448
Name: count, dtype: int64

至此，数据基本清理干净，并已把数据中相关缺失项已补充完全，并重新新建了一个clean_data.csv文件用于后续的分析

In [46]:
cleaned_data = original_data.copy()
cleaned_data.to_csv('../data/cleaned_salary_data.csv', index=False)

## 数据分析

根据清洗后的数据，我想分析一下性别、教育水平、工作年限和薪资的相关性